In [1]:
%pip install -q --upgrade pip
# this model is not supported in release, so we have to install nightly version
%pip uninstall -q -y openvino
%pip install -q openvino-nightly
# other dependencies
%pip install -q torch "transformers>=4.31.0"
%pip install -q "gradio>=4.0.2"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import TapasForQuestionAnswering
from transformers import TapasTokenizer
from transformers import pipeline
import pandas as pd

2023-11-09 22:54:53.482929: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 22:54:54.278161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 22:54:54.278398: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 22:54:54.281381: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 22:54:54.612232: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [3]:
# import pandas as pd
# from io import StringIO
# import requests
# cid = 'QmRTR2xVuxbVZ2umscMraQp4UsUwuCS2VHeJZNTKSi5zKL'
# ipfs_gateway_url = f'https://ipfs.io/ipfs/{cid}'
# response = requests.get(ipfs_gateway_url)
# csv_content = response.text
# table = pd.read_csv(StringIO(csv_content))
# print(table)



model = TapasForQuestionAnswering.from_pretrained('google/tapas-large-finetuned-wtq')
tokenizer = TapasTokenizer.from_pretrained("google/tapas-large-finetuned-wtq")
table = pd.read_csv('../dataset/symptom_and_medicine.csv', delimiter=",")
table = table.astype(str)
question = "medicine for fever and nausea?"
# print(table)



In [24]:
tqa = pipeline(task="table-question-answering", model=model, tokenizer=tokenizer)
result = tqa(table=table, query=question)
print(f"The answer is {result['cells'][0]}")

The answer is  Paracetamol


In [5]:
from pathlib import Path

import openvino as ov


# Define the input shape
batch_size = 1
sequence_length = 29

# Modify the input shape of the dummy_input dictionary
dummy_input = {
    "input_ids": torch.zeros((batch_size, sequence_length), dtype=torch.long),
    "attention_mask": torch.zeros((batch_size, sequence_length), dtype=torch.long),
    "token_type_ids": torch.zeros((batch_size, sequence_length, 7), dtype=torch.long),
}


ov_model_xml_path = Path('../../python_backend/trained_models/question_answering/qa.xml')

if not ov_model_xml_path.exists():
    ov_model = ov.convert_model(
        model,
        example_input=dummy_input
    )
    ov.save_model(ov_model, ov_model_xml_path)

In [6]:
import ipywidgets as widgets

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

In [21]:
import json

inputs = tokenizer(table=table, queries=question, padding="max_length", return_tensors="pt")

compiled_model = core.compile_model(ov_model_xml_path, device.value)


In [31]:

result = compiled_model((inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"]))


In [32]:
logits = result[0]
logits_aggregation = result[1]


predictions = tokenizer.convert_logits_to_predictions(inputs, torch.from_numpy(result[0]))
answer_coordinates_batch = predictions[0]
aggregators = {}
aggregators_prefix = {}
answers = []
for index, coordinates in enumerate(answer_coordinates_batch):
    cells = [table.iat[coordinate] for coordinate in coordinates]
    aggregator = aggregators.get(index, "")
    aggregator_prefix = aggregators_prefix.get(index, "")
    answer = {
        "answer": aggregator_prefix + ", ".join(cells),
        "coordinates": coordinates,
        "cells": [table.iat[coordinate] for coordinate in coordinates],
    }
    if aggregator:
        answer["aggregator"] = aggregator

    answers.append(answer)

print(answers[0]["cells"][0])

 Paracetamol


In [33]:
# from transformers import TapasConfig


# # get config for pretrained model
# config = TapasConfig.from_pretrained('google/tapas-large-finetuned-wtq')



# class TapasForQuestionAnswering(TapasForQuestionAnswering):  # it is better to keep the class name to avoid warnings
#     def __init__(self, ov_model_path):
#         super().__init__(config)  # pass config from the pretrained model
#         self.tqa_model = core.compile_model(ov_model_path, device.value)

#     def forward(self, input_ids, *, attention_mask, token_type_ids):
#         results = self.tqa_model((input_ids, attention_mask, token_type_ids))

#         return torch.from_numpy(results[0]), torch.from_numpy(results[1])


# compiled_model = TapasForQuestionAnswering(ov_model_xml_path)
# tqa = pipeline(task="table-question-answering", model=compiled_model, tokenizer=tokenizer)

# print(tqa(table=table, query=question)["cells"][0])